#  EV OPTIMIZATION 

In [26]:
import pandas as pd
import numpy as np
import numpy as np
from scipy import interpolate
import copy

import pandas as pd
import numpy as np
import random
import copy
import math

predictions = pd.read_csv("/Users/chemago/Research/Projects/Shell_AI2022_Challenge/Code/BaselinePrediction.csv")
EmptyMatrix = pd.read_csv("/Users/chemago/Research/Projects/Shell_AI2022_Challenge/Code/DSijMatrixZero.csv")
# for i in range(100):
#     EmptyMatrix[str(i)]  = EmptyMatrix[str(i)].round(2)
# EmptyMatrix.drop(["2018"])
# EmptyMatrix.to_csv
DistanceMatrix = pd.read_csv("/Users/chemago/Research/Projects/Shell_AI2022_Challenge/Code/DistanceMatrix.csv")
InfrasMatrix = pd.read_csv("/Users/chemago/Research/Projects/Shell_AI2022_Challenge/Code/EVinfrastructureZero.csv")
SupplyPointsDistanceMatrix = pd.read_csv("/Users/chemago/Research/Projects/Shell_AI2022_Challenge/Code/SupplyPointsDistanceMatrix.csv")
InfrasMatrix['currentFreeSlots'] = InfrasMatrix['FreeSlots2019']
InfrasMatrix['SMax']  = InfrasMatrix['FreeSlots2019']*400  + InfrasMatrix['chargingCapacity']
predictions['predDemand'] = predictions['2019'] # round(predictions['2019'],1)
predictions['predDemand'] = predictions['predDemand']#.astype(int)
#---------------------- OPTIMIZATION SECTION ----------------------------------------------------

InfrasMatrix['CurrentCapacity'] = InfrasMatrix['chargingCapacity']
# SupplyPointsDistances = InfrasMatrix[["supply_point_index",	"x_coordinate",	"y_coordinate"]]
# SupplyPointsDistances


#------------------ COMPUTE NEAREST NEIBORS OF SUPPLY POINTS--------------------------------------

NBS = [np.argsort(SupplyPointsDistanceMatrix[str(i)].to_list())[1:4] for i in range(100)]

In [27]:
dataset = predictions[['201'+str(i) for i in range(9)]]
exPdata =  dataset.T
exPdata

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
2010,0.352242,0.325940,0.373752,0.420686,0.475621,0.380156,0.449348,0.470071,0.537307,0.445734,...,0.114351,0.075160,0.081059,0.083821,0.158356,0.171015,0.041716,0.100895,0.155353,0.209372
2011,0.667932,0.591964,0.591890,0.584055,0.647940,0.725189,0.730466,0.866474,0.833661,0.775279,...,0.185420,0.131877,0.138802,0.158354,0.376271,0.334565,0.061741,0.180352,0.290825,0.340185
2012,0.958593,0.862652,0.969733,0.906547,0.981544,1.076054,1.220484,1.615454,1.622691,1.316209,...,0.489806,0.298811,0.227962,0.393987,0.670141,0.556055,0.131291,0.296299,0.557803,0.749491
2013,2.911901,2.589068,2.641432,2.378577,2.665400,2.762618,2.655187,3.273061,3.366533,3.795858,...,1.285151,0.819191,0.604009,0.966916,1.584710,1.373291,0.386540,0.705373,1.516066,1.904285
2014,4.338274,4.196034,3.541772,3.888121,4.218711,4.220747,5.333594,5.442867,5.973267,4.781361,...,2.168647,1.086649,1.012104,1.703340,2.272350,1.837586,0.755846,1.300220,2.399426,2.775772
2015,6.561995,5.745551,5.469161,5.846089,6.776609,6.760395,8.620496,8.681284,10.033183,9.277640,...,3.320948,1.427004,1.158568,2.083558,3.635703,2.517146,0.941116,1.608609,2.719197,3.404641
2016,8.454417,8.753195,8.414627,9.083868,8.851107,9.331196,11.406751,11.135352,13.210567,10.992479,...,4.336336,1.924844,1.350226,2.802044,5.880033,3.352280,1.107797,1.822806,4.494515,4.574922
2017,10.595324,11.126995,10.115336,12.424885,11.731131,14.743943,13.115854,14.708272,17.291626,17.386210,...,6.095297,2.054926,1.598313,3.284002,7.672174,4.149888,1.309479,2.333681,6.096858,6.301078
2018,13.119572,12.020091,14.018254,15.012302,16.355563,17.559998,19.696415,19.367417,21.899847,21.765241,...,6.625857,3.612612,2.643128,4.127373,7.833432,5.426193,2.057450,3.218519,6.262574,6.860939


In [28]:
InfrasMatrix

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS,SFtotal,FreeSlots2019,SCScap,FCScap,chargingCapacity,InfraCost,SMax,currentFreeSlots,CurrentCapacity
0,0,50.163110,19.412014,23,5,3,8,15,1000,1200,2200,9.5,8200,15,2200
1,1,37.336451,58.119225,27,4,7,11,16,800,2800,3600,14.5,10000,16,3600
2,2,46.709232,57.525650,31,6,14,20,11,1200,5600,6800,27.0,11200,11,6800
3,3,30.528626,55.379835,26,5,5,10,16,1000,2000,3000,12.5,9400,16,3000
4,4,51.521781,35.116755,32,11,6,17,15,2200,2400,4600,20.0,10600,15,4600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,4,7,17,600,1600,2200,9.0,9000,17,2200
96,96,30.318396,33.388335,32,5,10,15,17,1000,4000,5000,20.0,11800,17,5000
97,97,36.218839,22.235766,32,4,14,18,14,800,5600,6400,25.0,12000,14,6400
98,98,42.936915,38.122442,28,7,5,12,16,1400,2000,3400,14.5,9800,16,3400


In [29]:
InfrasMatrix["AllFCS"] = InfrasMatrix["currentFreeSlots"]*1.5 + InfrasMatrix["InfraCost"]
InfrasMatrix["AllRCS"] = InfrasMatrix["currentFreeSlots"]*2 + + InfrasMatrix["InfraCost"]

InfrasMatrix

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS,SFtotal,FreeSlots2019,SCScap,FCScap,chargingCapacity,InfraCost,SMax,currentFreeSlots,CurrentCapacity,AllFCS,AllRCS
0,0,50.163110,19.412014,23,5,3,8,15,1000,1200,2200,9.5,8200,15,2200,32.0,39.5
1,1,37.336451,58.119225,27,4,7,11,16,800,2800,3600,14.5,10000,16,3600,38.5,46.5
2,2,46.709232,57.525650,31,6,14,20,11,1200,5600,6800,27.0,11200,11,6800,43.5,49.0
3,3,30.528626,55.379835,26,5,5,10,16,1000,2000,3000,12.5,9400,16,3000,36.5,44.5
4,4,51.521781,35.116755,32,11,6,17,15,2200,2400,4600,20.0,10600,15,4600,42.5,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,4,7,17,600,1600,2200,9.0,9000,17,2200,34.5,43.0
96,96,30.318396,33.388335,32,5,10,15,17,1000,4000,5000,20.0,11800,17,5000,45.5,54.0
97,97,36.218839,22.235766,32,4,14,18,14,800,5600,6400,25.0,12000,14,6400,46.0,53.0
98,98,42.936915,38.122442,28,7,5,12,16,1400,2000,3400,14.5,9800,16,3400,38.5,46.5


In [30]:
sum(InfrasMatrix.AllFCS), sum(InfrasMatrix.AllRCS)

(3465.0, 4120.5)

# PREDICTOR 

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


def Ploypredict(data, n_train: int = 8, col: int = 0):

    dataS =  data[[col]].copy(deep=True)
    #-----------Train and Test Data -----------------------
    x =  np.array([(i+1) for i in range(n_train)]).reshape(-1,1)     # date
    y = dataS.head(n_train)[col].values.reshape(1,-1)[0]   # demand

    #----------TRANSFORM DATA ------------------------------
    transformer = PolynomialFeatures(degree=2, include_bias=False)
    transformer.fit(x)
    
    x_ = transformer.transform(x)
    #print(f' TRAINING RANGE {x}', )
    #-------------DEFINNE MODEL----------------------------

    model = LinearRegression().fit(x_, y)

    r_sq = model.score(x_, y)
    # print(f"coefficient of determination: {r_sq}")

    #------------MAKE PREDICTIONS-------------------------
    x_pred = []
    pr = n_train + 1
    while pr <= 11:
        x_pred.append([pr])
        pr += 1

    x_pred_ = transformer.transform(x_pred)
    # print(f" INSTANCES TO PREDICT :::::::: {x_pred_}")
    y_pred = model.predict(x_pred_)
    # print(f" PREDICTED RESPONSE :::::::::: {y_pred}")

    #------ERROR OF CURRENT PREDICTOR----------------------
    ValX = dataS.values[n_train:].reshape(-1,)
    # print(ValX, y_pred)
    errors =  []
    for i,trueX in enumerate(ValX):
        try:
            error = abs(trueX-y_pred[i])
            errors.append(error)
        except:
            pass
    # print(f" PREDICTION ERRORS ::::::::::: {errors}")

    return sum(errors), y_pred[-3:]


# * Determine potential Supply Points for Each Demand Point

1. Find the optimal distance that chooses supply points for all demand points without voilating all constraints
    - constraints are it doesn't exceed total demand at given location
    - For the search area, the Upper bound is max distance and the lower point is min distance 

Allocate DSij based on weighted distance

- Compute Mean Shortest Distance Between Supply Points and Demand Points (closes infrastructure within the distance is assigned to Supply Point)

In [32]:
# Important Number to  find distribution of demand points to supply  points

def meanDist(data: pd.DataFrame):
    """ compute the average mean, std etc from the entire dataset from supply to demand point"""

    import statistics
    Distances = []
    SortData = copy.deepcopy(data)

    for demandPoints in range(data.shape[0]):

        shortestSupplyPoint = SortData.values[demandPoints]
        shortestSupplyPoint.sort()
        # print(shortestSupplyPoint)
        MinDistance = shortestSupplyPoint[0]
        # print(SupplyPs[0])
        Distances.append(MinDistance)

    stdev = statistics.pstdev(Distances) 
    mn  = statistics.mean(Distances)

    return mn + stdev, mn, mn - stdev, max(Distances)*4 , min(Distances)  # max(Distances) #sum(Distances)/len(Distances)

meanDist(DistanceMatrix)

(6.557258703065985,
 4.000654757593667,
 1.4440508121213487,
 61.14825196819532,
 0.0918525162685226)

# 1. Get Distance

In [33]:
import copy
SortData = copy.deepcopy(DistanceMatrix)

def getSupplyPoints(data: pd.DataFrame, Distance: int = 5):
    """ data: demand supply distance matrix
        Distance: min distance to determine supply point (can be optimized iteratively)
    
    Computes the Supply Point for a given demand point: for a demand point, which supply point will serve it
    data: distance matrix """

    try:
        MaxDistance = Distance
    except:
        MaxDistance = 0 # meanDist(data)[2]  # index zero for mean+std, 1 for mean
    SupplyPoints = []
    # SortData = copy.deepcopy(data)
    print(data.shape[1])

    for demandPoints in range(data.shape[0]):

        # shortestSupplyPoint = SortData.values[demandPoints]
        # shortestSupplyPoint.sort()
        # MaxDistance = shortestSupplyPoint[NumPoints]
        # print(MaxDistance)
        SupplyPs = np.where(data.values[demandPoints] <= MaxDistance)[0]
        if len(SupplyPs) == 0:
            SupplyPs = np.where(data.values[demandPoints] == data.values[demandPoints].min())[0]
        
        # SupplyPs = np.where(data.values[demandPoints] == data.values[demandPoints].min())[0]

        # print(SupplyPs[0])
        SupplyPoints.append(SupplyPs)

    return SupplyPoints


# Supply Count and Quantity

In [35]:
def SupplyQuantification(PotSupplyPoints, data = None):
    """ Computes the demands points for each supply point and the expected demand quantity"""
    
    Serve = []
    # Demand = []
    DPS = []

    for i in range(100):
        
        d = []
        for Index,j in enumerate(PotSupplyPoints):
            # print(j)
            if i in j:
                d.append(Index)
                # print("Demand Points ", Index, " for Supply Points ", i)
        Serve.append(len(d))
        DPS.append(d[:])
        # Demand.append([])
       # print("Number of demand Points ", len(DPS[i]), Serve[i], " for Supply Points ", i)
    
    return Serve, DPS


In [36]:
def totoalDemandInSP(PredictedDemand, Demandpoints4Supply, supplyPoints4Demand):
    """ Computes the Total Demand at a given suplly point """
    SPS = []
    for i in Demandpoints4Supply:
        count = 0
        for j in i:
            # print(supplyPoints4Demand[j])
            count += PredictedDemand[j]/len(supplyPoints4Demand[j])# divide by number of stations to supply
        SPS.append(count)
    return SPS

# Compute Infrastructure to Build Based On Demand

# Share Built Infrastrure to Neighbors

In [37]:
def GetCapacityToBuilD(InfrasMatrix, YearDemand, SupplyPointNeibors, Sharing = False, Proportion2Share = 1):
    """ 
        InfrasMatrix: contains information about supply points 
        YearDemand: determined demand at supply points
        SupplyPointNeibors: supply points closses to a given supply point
        Sharing: If capacity should be shared with neighbors
    
        DifCurrentCap: Capacity met for each demand point 
        DifCurrentMax: Capacity left to be met by adding Max capacity
        AddedCapMax: Capacity to hold to meet current demand

        """

    # YearDemand[i] is the determined demand at a supply point
    DiffCurrentCap = [InfrasMatrix.CurrentCapacity.values[i] - YearDemand[i] for i in range(len(InfrasMatrix))] 
    AddedCapMax = [InfrasMatrix.CurrentCapacity.values[i] + YearDemand[i] for i in range(len(InfrasMatrix))]
    # DifferenceCapMax = [InfrasMatrix.SMax.values[i] - YearDemand[i] for i in range(len(InfrasMatrix))]

    #------------------- FILLING REQUIRED DEMAND-------------------------------------------------------------------------------
    DemandToBuild = []
    Demand2Share = []
    for i,cap in enumerate(DiffCurrentCap):
        # CHeck if new infrastructure needs to be built
        if cap >= 0:
            DemandToBuild.append(0) # No need to build infrastructure here 
            Demand2Share.append(0)  
        else:
            # Check if max capaciity can hold demand
            # print(f" WILL BUILD NEW DEMAND FOR {i}")
            if AddedCapMax[i]  <= InfrasMatrix.SMax.values[i]:
                DemandToBuild.append( YearDemand[i] )         # the demand is enough for this capacity 
                Demand2Share.append(0)
            else:
                DemandToBuild.append( InfrasMatrix.SMax.values[i] -  InfrasMatrix.CurrentCapacity.values[i]) # build to the max
                Demand2Share.append( AddedCapMax[i] - InfrasMatrix.SMax.values[i] ) # demand to share with neighbours dealt with by OverCapacity

    #--------------------DEALING WITH OVERCAPACITY------------------------------------------------------------------------------

    if Sharing:
        for i,D2Share in enumerate(Demand2Share):
            # check if there is demand to share
            if D2Share > 0:
                NBs = SupplyPointNeibors[i][1:]
                for k in NBs:
                    # calculate acceptable demand 
                    # print(f" Sharing Demand of Point {i} WITH POINT {k}")
                    D2Take = InfrasMatrix.SMax.values[k] - ( DemandToBuild[k] +  InfrasMatrix.CurrentCapacity.values[k] )
                    # Check if Neigbor meets Capacity
                    if D2Take >= D2Share:
                        # Add Demand From Current Station ====  SHARE ALL DEMAND =====
                        Proportion2Take = Proportion2Share
                        ToBuild = DemandToBuild[k] + D2Share*Proportion2Take #*0.75
                        DemandToBuild[k] = ToBuild
                        # print(" SHARED AND TERMINATING ")
                        break
                    else:
                        # break
                        Proportion2Take = Proportion2Share
                        # ToBuild = DemandToBuild[k] + D2Take
                        ToBuild = DemandToBuild[k] + D2Take*Proportion2Take
                        # print(f"{ToBuild} ========== {InfrasMatrix.SMax.values[k]} FOR {i} by {k}")
                        DemandToBuild[k] = ToBuild
                        # D2Share =  D2Share -  D2Take*Proportion2Take # /2.5
                        # break
    
    return DemandToBuild

# Build New Infrastructure

In [38]:
from ortools.algorithms import pywrapknapsack_solver
import random

def BuildInfra(parameters: list, freeslots: list, InfrasMatrix: pd.DataFrame, Year = 2019):
    """ determines the number of charging stations to build at each supply location 
    
    
    """

    NewInfrasInfo = copy.deepcopy(InfrasMatrix)
    # Capacities = NewInfrasInfo.FreeSlots2019
    Cost = 0
    # Costs = []
    # Stations  =  []
    SCS  = []
    FCS = []
    #-----------------ALLOCATION-------------------------------------------------------------------
    for station, freeSLOTS in enumerate(freeslots):
        #---------------SET PARAMETERS-------------------------------------------------------------
        Capacity = parameters[station] # max weight of items 
        # Capacity = (int(math.ceil(parameters[station]/ 100.0)) * 100  ) - 200 # round to the nearest two hundred

        # Items = ["FCS" for _ in range(int(freeSLOTS))] + ["SCS" for _ in range(int(freeSLOTS))]
        # Items = ["FCS" for _ in range(int(freeSLOTS*0.75))] + ["SCS" for _ in range(int(freeSLOTS*0.25))]
        # Items = ["FCS" for _ in range(int(freeSLOTS*0.9))] + ["SCS" for _ in range(int(freeSLOTS*0.1))]
        Items = ["FCS" for _ in range(int(freeSLOTS))] + ["SCS" for _ in range(int(freeSLOTS))]



        # random.shuffle(Items)
        # values = [1.5 if item == "FCS" else 0.55 for item in Items]
        values = [1.5 if item == "FCS" else 0.95 for item in Items]

        # values = [1 for _ in Items]
        weights = [[400 if item == "FCS" else 200 for item in Items]]
        capacities = [Capacity]

        #---------------SOLVE PACKING PROBLEM----------------------------------------------------------------
        solver = pywrapknapsack_solver.KnapsackSolver(
            pywrapknapsack_solver.KnapsackSolver.
            KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER, 'KnapsackExample')

        solver.Init(values, weights, capacities)
        #computed_value = 
        solver.Solve()
        # print(weights[0], solver.BestSolutionContains(0))
        packed_items = [Items[x] for x in range(0, len(weights[0]))
                        if solver.BestSolutionContains(x)]

        # print(packed_items, "||||||  ITEMS  |||| ", Items)
        stationCost = sum([1.5 if item == "FCS" else 1 for item in packed_items])
        Cost += stationCost

        FCS.append(packed_items.count('FCS'))
        SCS.append(packed_items.count('SCS'))
        
        #-------------- BUILD AT LEAST ONE SLOW CHARGING STATION
        # if (SCS[station] == 0) and (FCS[station] > 10):
        #     # print("ADDING SLOW COZ THERE IS NONE")
        #     SCS[station] = 1
        #     FCS[station] = FCS[station] - 1

        if FCS[station] + SCS[station] > int(freeSLOTS):
            SCS[station] = 0
            print(f"INITIAL PACKING   === FCS = {packed_items.count('FCS')} SCS = {packed_items.count('SCS')} NUMBER OF PACKED ITEMS {len(packed_items)}, MAX REQUIRE {int(freeSLOTS)}")
            print(f"CORRECTED PACKING === FCS = {FCS[station]} SCS = {SCS[station]} NUMBER OF PACKED ITEMS {FCS[station] + SCS[station]}, MAX REQUIRE {int(freeSLOTS)}")
        # Costs.append(stationCost)
            
    #---------------UPDATE INFRA DATA------------------------------------------------
    NewStations = pd.DataFrame(data = {'FCS':FCS, 'SCS':SCS})
    # print(NewStations)
    # NewStations = NewStations.fillna(0)
    NewInfrasInfo['NewChargersFCS'] = NewStations['FCS'].astype(int)
    NewInfrasInfo['NewChargersSCS'] = NewStations['SCS'].astype(int)

    # print("NEW STATIONS FCS", NewInfrasInfo['NewChargersFCS'].values)
    # print("NEW STATIONS SCS", NewInfrasInfo['NewChargersSCS'].values)

    if Year == 2020:
        #-------------SECOND TIME i.e  2020 COMPUTATION-------------------------------
        NewInfrasInfo['CurrentSCS'] = NewInfrasInfo['CurrentSCS'] + NewInfrasInfo['NewChargersSCS'] # NewStations['SCS']
        NewInfrasInfo['CurrentFCS'] = NewInfrasInfo['CurrentFCS'] + NewInfrasInfo['NewChargersFCS'] # 
        NewInfrasInfo['CurrentCapacity'] = NewInfrasInfo['CurrentSCS']*200 + NewInfrasInfo['CurrentFCS']*400
        NewInfrasInfo['CurrentCost'] = NewInfrasInfo['CurrentSCS'] + NewInfrasInfo['CurrentFCS']*1.5
        NewInfrasInfo['currentFreeSlots'] = NewInfrasInfo['total_parking_slots'] - (NewInfrasInfo['CurrentSCS'] + NewInfrasInfo['CurrentFCS'])
        print(" ========== COMPLETED BUILDING INFTASTRUCTURE for 2020 ================ ")
    if Year == 2019:
        #-------------FIRST TIME i.e  2019 COMPUTATION-------------------------------
        NewInfrasInfo['CurrentSCS'] = NewInfrasInfo['existing_num_SCS'] + NewStations['SCS']
        NewInfrasInfo['CurrentFCS'] = NewInfrasInfo['existing_num_FCS'] + NewStations['FCS']
        NewInfrasInfo['CurrentCapacity'] = NewInfrasInfo['CurrentSCS']*200 + NewInfrasInfo['CurrentFCS']*400
        NewInfrasInfo['CurrentCost'] = NewInfrasInfo['CurrentSCS'] + NewInfrasInfo['CurrentFCS']*1.5
        NewInfrasInfo['currentFreeSlots'] = NewInfrasInfo['total_parking_slots'] - (NewInfrasInfo['CurrentSCS'] + NewInfrasInfo['CurrentFCS'])
        print(" ========== COMPLETED BUILDING INFTASTRUCTURE for 2019 ================ ")

    Checker = NewInfrasInfo.currentFreeSlots.values < 0
    for i,j in enumerate(Checker):
        if j == True:
            print("CAPACITY VOILATED")
            print( " VOILATED STATION CAPACITY AT STATION", i)
            print( NewInfrasInfo.SMax.values[i], " BUILD IS MORE THAN  AVIALABLE ", NewInfrasInfo.CurrentCapacity.values[i])
            print( NewInfrasInfo.CurrentFCS.values[i] + NewInfrasInfo.CurrentSCS.values[i], " BUILD IS MORE THAN  AVIALABLE ", NewInfrasInfo.total_parking_slots.values[i])

    #---------------UPDATE INFRA DATA----------------------------------------------------------------

    # print( pd.Series(NewInfrasInfo.currentFreeSlots.values < 0).value_counts)
    #---------------SOLVE ALLOCATION WITH RESPECT TO INFRACAPACITY------------------------------------------

    print(" ========== ADDED COST FROM BUILT INFRASTRUCTURE  == ", Cost, "===== TOTAL INFRASTRUCTURE COST === ", NewInfrasInfo.CurrentCost.sum())
    return NewInfrasInfo.CurrentCost.sum(), NewInfrasInfo

# ALLOCATION

In [39]:
# Import PuLP modeler functions
from pulp import *
import random
import numpy as np
import pandas as pd

def SupplyDemansAllocation(InfrasMatrix: pd.DataFrame, DistanceMatrix: pd.DataFrame, predictions: pd.DataFrame, AllocationMatrix:pd.DataFrame):

    # Creates a list of all the supply nodes
    SupplyPoints = ["SP_" + str(SP) for SP in range(100)]
    # Creates a dictionary for the number of units of supply for each supply node
    SupplyCapacities = {"SP_" + str(SP) : SPCap for SP, SPCap in enumerate(InfrasMatrix.CurrentCapacity)}
    # Creates a list of all demand nodes
    DemandPoints = [str(demandPoint) for demandPoint in range(DistanceMatrix.shape[0])]
    # Creates a dictionary for the number of units of demand for each demand node
    Demands = {str(demandPoint): DEmands for demandPoint, DEmands in enumerate(predictions.predDemand.values)}

    distanceMatrix = DistanceMatrix.T.values
    # The cost data is made into a dictionary
    distanceCost = makeDict([SupplyPoints,DemandPoints],  distanceMatrix, 0)

    # Creates the 'prob' variable to contain the problem data
    Prob = LpProblem("Beer Distribution Problem", LpMinimize)

    # Creates a list of tuples containing all the possible routes for transport
    Roads = [(w, b) for w in SupplyPoints for b in DemandPoints]

    # A dictionary called 'Vars' is created to contain the referenced variables(the routes)
    Vars = LpVariable.dicts("Road", (SupplyPoints, DemandPoints), 0, None, LpContinuous)
    # Vars = LpVariable.dicts("Road", (SupplyPoints, DemandPoints), 0, tuple(InfrasMatrix.CurrentCapacity), LpContinuous)

    # The objective function is added to 'prob' first
    Prob += (
        lpSum([Vars[w][b] * distanceCost[w][b] for (w, b) in Roads]),
        "Sum_of_Transporting_Costs",
    )

    # The supply maximum constraints are added to prob for each supply node (warehouse)
    for c in SupplyPoints:
        Prob += (
            lpSum([Vars[c][b] for b in DemandPoints]) <= SupplyCapacities[c],
            "Sum_of_Products_out_of_Warehouse_%s" % c,
        )

    # The demand minimum constraints are added to prob for each demand node (bar)
    for d in DemandPoints:
        Prob += (
            lpSum([Vars[w][d] for w in SupplyPoints]) == Demands[d],
            "Sum_of_Products_into_Bar%s" % d,
        )

    # The problem data is written to an .lp file
    Prob.writeLP("EVDistributionProblem.lp")

    # The problem is solved using PuLP's choice of Solver
    # Prob.solve(GLPK(msg = True))
    Prob.solve(PULP_CBC_CMD(msg=0))

    # The status of the solution is printed to the screen
    # print("Status:", LpStatus[Prob.status])

    # Each of the variables is printed with it's resolved optimum value
    # for v in Prob.variables():
    #     print(v.name, "=", v.varValue)

    # The optimised objective function value is printed to the screen
    
    print("Total Travel Cost = ", value(Prob.objective), " STATUS ",  Prob.solver)

    # ------------ Fill in the matriix --------------------------------
    DSIJ = copy.deepcopy(AllocationMatrix)
    for v in Prob.variables(): # Prob.variables():
        if v.varValue > 0 :
            # print(v.name, " has value " , v.varValue)
            rc = v.name.split("_")
            row, col = int(rc[3]), rc[2]
            DSIJ.at[row, col] = v.varValue - 0.001
            # DSIJ.at[row, col] = round(v.varValue,2)
            # DSIJ.at[row, col] = int(v.varValue)  # round(v.varValue, 1)
    # ----------- CHECK FOR CONSTRAINT VOILATIONS -----------------------
    for i in range(100):
        # if DSIJ[str(i)].sum() > InfrasMatrix['CurrentCapacity'][i]:
        if DSIJ[str(i)].sum() > InfrasMatrix['CurrentCapacity'][i]:
            print( " VOILATED SUM OF FRACTIONAL CONSTRAINTS OF MAX SUPPLY FOR DEMAND POINT ", i)
            print(DSIJ[str(i)].sum(),  " Greater Than ",  InfrasMatrix['CurrentCapacity'][i], " Should be Less Than ")

    predCheck = predictions.predDemand.values
    PrecisionPredictions = []
    solCheck = DSIJ.values
    for i in range(len(DemandPoints)):
        PrecisionPredictions.append(solCheck[i][2:].sum())
        # print(predCheck[i], solCheck[i][2:])
        if round(predCheck[i]) != round(solCheck[i][2:].sum()):
        # if abs (predCheck[i] - solCheck[i][2:].sum()) < 0.01 :
            print( " VOILATED SUM OF FRACTIONAL CONSTRAINTS OF DEMAND FOR DEMAND POINT ", i)
            print( predCheck[i], " NOT EQUAL TO  ", solCheck[i][2:].sum())
        
    return value(Prob.objective), DSIJ, PrecisionPredictions

In [40]:
3.22343 + 12.34

15.56343

In [58]:
# predictions['predDemand'] = predictions['2017'] * 1.32+predictions['2018'] * 0.15

In [41]:
TrainSamples = 9
Total = 0
YValues = []
for i in range(exPdata.shape[1]):
    Results = Ploypredict(exPdata, n_train=TrainSamples, col=i)
    # print(Results[0])
    Total += Results[0]
    YValues.append(Results[1])

BasePredictions = pd.DataFrame(YValues)
print(f" ERROR {(abs(BasePredictions[0] - predictions['2018']).sum())}")
print(Total)
BasePredictions

 ERROR 93431.95628704267
0


,0,1
0,16.257476,19.498171
1,15.728390,18.840683
2,17.383617,21.375862
3,19.569853,24.151719
4,20.224604,24.958468
...,...,...
4091,6.528385,7.824115
4092,2.359797,2.845685
4093,3.752876,4.470594
4094,8.235054,9.845793


In [42]:
pd.DataFrame(BasePredictions[0]-predictions['2018'])[0].sum()

93431.95628704267

In [43]:
if TrainSamples == 9:
    predictions['2019'] = BasePredictions[0]
    predictions['2020'] = BasePredictions[1]
else:
    predictions['2018+'] = BasePredictions[0]
    predictions['2019'] = BasePredictions[1] 
    predictions['2020'] = BasePredictions[2] 
    
PredError  = (abs(BasePredictions[0] - predictions['2018']).sum())
PredError


93431.95628704267

In [23]:
distance = 5
for i in range(10):
    PotSupplyPoints = getSupplyPoints(DistanceMatrix, distance)
    Count, Stations = SupplyQuantification(PotSupplyPoints)

    # predictions['2019'] = predictions['2017'] * 1.32 + predictions['2018'] * 0.15  # predictions
    PredictedDemand = predictions['2019'].values#.astype(int)
    SupplyPointDemand =  totoalDemandInSP(PredictedDemand, Stations, PotSupplyPoints)

    ShareBuildDemand = GetCapacityToBuilD(InfrasMatrix, SupplyPointDemand, NBS, Sharing = False , Proportion2Share = 0.99)
    predictions['predDemand'] = predictions['2019']#.astype(int)
   
    # predictions['predDemand'] = predictions['predDemand'].astype(int)
    print(ShareBuildDemand)
    Cost, Infrastructure2019  =  BuildInfra(ShareBuildDemand, InfrasMatrix.currentFreeSlots, InfrasMatrix, Year = 2019)
    Score, Allocation2019, _  = SupplyDemansAllocation(Infrastructure2019, DistanceMatrix, predictions, AllocationMatrix=EmptyMatrix)
    print("=========== ERROR IS, == " , 90*(Score + Cost + (25*PredError) )/15000000 , "=========")
    break
    distance *= distance
Infrastructure2019

100
[2791.7301172746284, 4158.125082645845, 0, 3417.493618892006, 0, 0, 0, 3943.7210716717354, 4601.305286916345, 4000, 3200, 3200, 4800, 5578.818585396757, 0, 5200, 2800, 1200, 5749.105900767635, 0, 0, 0, 3275.6953667708194, 0, 0, 0, 5200, 3847.89072662751, 0, 7440.931608042091, 3555.470246916129, 0, 0, 5748.937379843626, 5200, 2400, 0, 2000, 0, 3930.026599146325, 0, 3641.9469452421, 5394.813613831033, 0, 0, 4994.214371351711, 0, 3266.7455997658676, 0, 0, 6000, 1287.3220464043739, 0, 0, 0, 1128.7376834273034, 0, 0, 0, 0, 0, 0, 0, 0, 4729.889815645742, 7200, 4000, 0, 0, 7600, 0, 0, 4659.764877131702, 3602.9206637216025, 2800, 0, 1600, 0, 0, 5944.680115089211, 0, 8400, 3600, 0, 0, 0, 5600, 3977.2628774336504, 3361.896172529748, 0, 4000, 0, 4000, 0, 0, 3568.7514027855314, 6800, 5600, 3798.7007225215975, 0]
 ========== COMPLETED BUILDING INFTASTRUCTURE for 2019 ================ 
 ========== ADDED COST FROM BUILT INFRASTRUCTURE  ==  794.5 ===== TOTAL INFRASTRUCTURE COST ===  2293.0


/Users/chemago/opt/anaconda3/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Total Travel Cost =  1583425.6981481784  STATUS  <pulp.apis.coin_api.PULP_CBC_CMD object at 0x7fbe2aa0f070>
 VOILATED SUM OF FRACTIONAL CONSTRAINTS OF DEMAND FOR DEMAND POINT  184
43.500676368216034  NOT EQUAL TO   43.499676
 VOILATED SUM OF FRACTIONAL CONSTRAINTS OF DEMAND FOR DEMAND POINT  3528
33.50080158899706  NOT EQUAL TO   33.499802
 VOILATED SUM OF FRACTIONAL CONSTRAINTS OF DEMAND FOR DEMAND POINT  4023
5.500388337103482  NOT EQUAL TO   5.4993883
=========== ERROR IS, ==  23.52910563194547 =========


,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS,SFtotal,FreeSlots2019,SCScap,FCScap,chargingCapacity,InfraCost,SMax,currentFreeSlots,CurrentCapacity,NewChargersFCS,NewChargersSCS,CurrentSCS,CurrentFCS,CurrentCost
0,0,50.163110,19.412014,23,5,3,8,15,1000,1200,2200,9.5,8200,9,4600,6,0,5,9,18.5
1,1,37.336451,58.119225,27,4,7,11,16,800,2800,3600,14.5,10000,6,7600,10,0,4,17,29.5
2,2,46.709232,57.525650,31,6,14,20,11,1200,5600,6800,27.0,11200,11,6800,0,0,6,14,27.0
3,3,30.528626,55.379835,26,5,5,10,16,1000,2000,3000,12.5,9400,8,6200,8,0,5,13,24.5
4,4,51.521781,35.116755,32,11,6,17,15,2200,2400,4600,20.0,10600,15,4600,0,0,11,6,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,45.471204,20.999414,24,3,4,7,17,600,1600,2200,9.0,9000,9,5400,8,0,3,12,21.0
96,96,30.318396,33.388335,32,5,10,15,17,1000,4000,5000,20.0,11800,0,11800,17,0,5,27,45.5
97,97,36.218839,22.235766,32,4,14,18,14,800,5600,6400,25.0,12000,0,12000,14,0,4,28,46.0
98,98,42.936915,38.122442,28,7,5,12,16,1400,2000,3400,14.5,9800,7,7000,9,0,7,14,28.0


In [25]:
2293.0 - 794

1499.0

In [63]:
# ASS = Allocation2019[[str(i) for i in range(100)]].values
ASS = Allocation2019[[str(i) for i in range(100)]].T.values

for k,ASH in enumerate(ASS):
    loc  =  np.nonzero(ASH)[0]
    if ASH.sum() >= Infrastructure2019.SMax[k]:
        #print(len(ASH))
        print(ASH.sum()-0.001, " UNEQUAL ", Infrastructure2019.SMax[k], random.choice(loc), ASH[random.choice(loc)]) 


In [64]:
def genSubmisionData(InfrasSolution, DS, year):

    # DS = [InfrasSolution.values[i][2:] for i in range(len(InfrasSolution))]
    DS = DS[[str(i) for i in range(100)]]

    # ----------- Infrastructure Data --------------------------------------------------------------------------------
    SPSdata =  pd.DataFrame()
    SPSdata['year'] = [year for _ in range(200)]
    SPSdata['data_type'] = ['SCS' for _ in range(100)] + ['FCS' for _ in range(100)]
    SPSdata['demand_point_index'] = None
    SPSdata['supply_point_index'] = [supplyPoint for supplyPoint in range(100)] + [supplyPoint for supplyPoint in range(100)]
    SPSdata['value'] = [value for value in InfrasSolution.CurrentSCS] + [value for value in InfrasSolution.CurrentFCS]

    # -------------Demand Data ------------------------------------------------------------------------------------------
    DPSdata =  pd.DataFrame()
    for i in range(4096):
        if i == 0:
            DPSdata['year'] = [year for _ in range(100)]
            DPSdata['data_type'] = ['DS' for _ in range(100)] 
            DPSdata['demand_point_index'] = i
            DPSdata['supply_point_index'] = [supplyPoint for supplyPoint in range(100)]
            DPSdata['value'] = DS.values[i]#.round(2)
        else:
            addDPSdata =  pd.DataFrame()
            addDPSdata['year'] = [year for _ in range(100)]
            addDPSdata['data_type'] = ['DS' for _ in range(100)] 
            addDPSdata['demand_point_index'] = i
            addDPSdata['supply_point_index'] = [supplyPoint for supplyPoint in range(100)]
            addDPSdata['value'] = DS.values[i]#.round(2)

            # DPSdata = DPSdata.append(addDPSdata, ignore_index=True)
            DPSdata = pd.concat([DPSdata,addDPSdata], ignore_index=True)

    return pd.concat([SPSdata,DPSdata], ignore_index=True)


Solution2019 = genSubmisionData(Infrastructure2019, Allocation2019, year=2019)

Solution2019

,year,data_type,demand_point_index,supply_point_index,value
0,2019,SCS,None,0,5.0
1,2019,SCS,None,1,4.0
2,2019,SCS,None,2,6.0
3,2019,SCS,None,3,5.0
4,2019,SCS,None,4,11.0
...,...,...,...,...,...
409795,2019,DS,4095,95,0.0
409796,2019,DS,4095,96,0.0
409797,2019,DS,4095,97,0.0
409798,2019,DS,4095,98,0.0


In [65]:
# 2020

In [25]:
distance = 5
for i in range(10):
    PotSupplyPoints = getSupplyPoints(DistanceMatrix, distance)
    Count, Stations = SupplyQuantification(PotSupplyPoints)

    # predictions['2020'] = predictions['2018'] * 1.32 + predictions['2019'] * 0.15  # predictions
    PredictedDemand = predictions['2020'].values#.astype(int)
    SupplyPointDemand =  totoalDemandInSP(PredictedDemand, Stations, PotSupplyPoints)
    ShareBuildDemand = GetCapacityToBuilD(Infrastructure2019, SupplyPointDemand, NBS, Sharing = False, Proportion2Share = 0.99)
    predictions['predDemand'] = predictions['2020']#.astype(int)
   
    # predictions['predDemand'] = predictions['predDemand'].astype(int)
    print(ShareBuildDemand)
    Cost, Infrastructure2020  =  BuildInfra(ShareBuildDemand, Infrastructure2019.currentFreeSlots, Infrastructure2019, Year = 2020)
    Score, Allocation2020, MatchingPredictions = SupplyDemansAllocation(Infrastructure2019, DistanceMatrix, predictions, AllocationMatrix=EmptyMatrix)
    print("===========ERROR IS, == " , 90*(Score + Cost + 25*PredError)/15000000 , "=========")
    break
    distance *= distance
Infrastructure2020

100


NameError: name 'Infrastructure2019' is not defined

In [67]:
Solution2020 = genSubmisionData(Infrastructure2020, Allocation2020, year = 2020)

In [68]:
FinalSubmissionFile  = pd.concat([Solution2019,  Solution2020], ignore_index=True)
FinalSubmissionFile['value'] = FinalSubmissionFile['value'].astype(float)
print(FinalSubmissionFile.dtypes)
FinalSubmissionFile.to_csv('Analytical EV Baseline0011.csv', index=False)

year                    int64
data_type              object
demand_point_index     object
supply_point_index      int64
value                 float64
dtype: object


In [69]:
FinalSubmissionFile

,year,data_type,demand_point_index,supply_point_index,value
0,2019,SCS,None,0,5.0
1,2019,SCS,None,1,4.0
2,2019,SCS,None,2,6.0
3,2019,SCS,None,3,5.0
4,2019,SCS,None,4,11.0
...,...,...,...,...,...
819595,2020,DS,4095,95,0.0
819596,2020,DS,4095,96,0.0
819597,2020,DS,4095,97,0.0
819598,2020,DS,4095,98,0.0


In [35]:
600/25

24.0

In [45]:
def relativeCostDS(CostIF, standardCostIF = 2441.5, costDS = 2163985.83):

    return (standardCostIF/CostIF)*costDS

    
relativeCostDS(4120.5) 

1282216.0912377138

In [ ]:
1499	,4032105
5588	,915180
3316	,1923029
2995	,2205302
2721	,1903332
3133	,2070283
3347	,1528166



In [65]:
def cost(IFcost, DScost):

    return IFcost + 24*DScost

cost(3347	,1528166)

36679331

# Infrastructure cost 

In [33]:
data = [1727
,4113
,2490
,2148
,2084
,2277
,2439
]

# Infrastructure
for i in data:
    print(random.uniform(1.3,1.4)*i)


2265.8506692470023
5588.181903724453
3316.979990348876
2995.65115541955
2721.463484230462
3133.453184882697
3347.683036496885


In [36]:
import random
data = [1514158
,887065
,153890
,411009
,470847
,157669
,10385
]

for i in data:
    print(random.uniform(0.9,1.1)*i)

1620303.1433074968
886744.2651791892
145745.70697371918
379099.95858677366
440496.86901906016
164674.9950495252
10183.528061701274


In [47]:
(2128/2293.0)*2441.5

2265.81421718273

In [48]:


CostIF2017 = {"NC" : 1499, "BA" : 3465, "PSO" : 2144, "GA" : 1935, "HA" : 1872, "ES" : 2128, "OURS" : 2293}

CostDS2017 = {"NC" : 2422144.83, "BA" : 1047848.51, "PSO" : 1693467.86, "GA" : 1876379.9, "HA" : 1939527.3, "ES" : 1706200.7, "OURS" : 1583425.69}

Cost2017 = {}
for algo in CostIF2017:
    print(f"Cost  for {algo} is {cost(CostIF2017[algo], CostDS2017[algo])}")
    Cost2017[algo] = cost(CostIF2017[algo], CostDS2017[algo])

Cost2017.values()


Cost  for NC is 58132974.92
Cost  for BA is 25151829.240000002
Cost  for PSO is 40645372.64
Cost  for GA is 45035052.599999994
Cost  for HA is 46550527.2
Cost  for ES is 40950944.8
Cost  for OURS is 38004509.56


dict_values([58132974.92, 25151829.240000002, 40645372.64, 45035052.599999994, 46550527.2, 40950944.8, 38004509.56])

In [49]:
TrueCostDs2017 = 1531354.69

ErrorCost2017 = [TrueCostDs2017 - Cozt for Cozt in CostDS2017.values()]
ErrorCost2017

[-890790.1400000001,
 483506.17999999993,
 -162113.17000000016,
 -345025.20999999996,
 -408172.6100000001,
 -174846.01,
 -52071.0]

In [50]:
CostIF2018 = {"NC" : 1499, "BA" : 4120.5, "PSO" : 2282.85, "GA" : 2060.31, "HA" : 1993.23, "ES" : 2265.81, "OURS" : 2441.5}

CostDS2018 = {"NC" : 3524597.33, "BA" : 1282216.09, "PSO" : 2314375.19, "GA" : 2564357.50, "HA" : 2650658.17, "ES" : 2331780.42, "OURS" : 2163985.83}

Cost2018 = {}
for algo in CostIF2018:
    print(f"Cost  for {algo} is {cost(CostIF2018[algo], CostDS2018[algo])}")
    Cost2018[algo] = cost(CostIF2018[algo], CostDS2018[algo])

Cost2018.values()

Cost  for NC is 84591834.92
Cost  for BA is 30777306.660000004
Cost  for PSO is 55547287.410000004
Cost  for GA is 61546640.31
Cost  for HA is 63617789.309999995
Cost  for ES is 55964995.89
Cost  for OURS is 51938101.42


dict_values([84591834.92, 30777306.660000004, 55547287.410000004, 61546640.31, 63617789.309999995, 55964995.89, 51938101.42])

In [51]:
TrueCostDs2018 = 2174385.54

ErrorCost2018 = [TrueCostDs2018 - Cozt for Cozt in CostDS2018.values()]
ErrorCost2018

[-1350211.79,
 892169.45,
 -139989.6499999999,
 -389971.95999999996,
 -476272.6299999999,
 -157394.8799999999,
 10399.709999999963]

In [29]:
import random
import numpy as np

fluctuation = {"NC" : [0.9, 1.4], "BA" : [0.9, 1.1], "PSO" : [0.95, 1.25], "GA" : [0.85, 1.25], "HA" : [0.8, 1.25], "ES" : [0.93, 1.08], "OURS" : [0.98, 1.02]}

In [45]:
costs = [29
,1
,18
,21
,22
,18
,16
]



for ind, algo in enumerate(fluctuation):
    test = []
    for i in range(100):
        tempcost = random.uniform(fluctuation[algo][0], fluctuation[algo][1])*costs[ind]
        test.append(tempcost)
    
    meanCost = np.array(test).mean()
    print(f"Mean cost for {algo} is {meanCost}")


Mean cost for NC is 32.39509749396933
Mean cost for BA is 1.012809666290449
Mean cost for PSO is 19.640199940617862
Mean cost for GA is 21.961266293502714
Mean cost for HA is 22.455126003521606
Mean cost for ES is 18.01722566045864
Mean cost for OURS is 15.999824296959014


# Average Travel Time

In [96]:
# TT2017 = CostDS2017

divisor = 1583425.69/10

distanace_result = 1583425.69/divisor

print(distanace_result)

def TravelTime(DScost, divisors):

    return DScost/divisors

for g in CostDS2017:

    print(g, (CostDS2018[g]/divisor) )

10.0
NC 22.259316318153207
BA 8.097734539092897
PSO 14.616253889375763
GA 16.194997442538654
HA 16.74002251409727
ES 14.726175246026228
OURS 13.66648175324224


TravelTime2017 = {NC 15.296864546892632
                    BA 6.617604581115518
                    PSO 10.694962641410726
                    GA 11.850129196779674
                    HA 12.248931618635037
                    ES 10.775375887705852
                    OURS 10.0}

TravelTime2018 = {NC 22.259316318153207
                    BA 8.097734539092897
                    PSO 14.616253889375763
                    GA 16.194997442538654
                    HA 16.74002251409727
                    ES 14.726175246026228
                    OURS 13.66648175324224}

WaitingTime2017 = {NC 28.99914090343789
                    BA 3.8240459168758947
                    PSO 18.146931188020044
                    GA 21.38608697206785
                    HA 22.414629252380582
                    ES 18.385507837901013
                    OURS 16.0}

WaitingTime2018 = {NC 28.99914090343789
                    BA 0.659403493710071
                    PSO 16.14196200004983
                    GA 19.41207509336993
                    HA 20.456901741833974
                    ES 16.381700230949956
                    OURS 13.994712286336107}

ChargingTime2017 = {NC 30
                    BA 13
                    PSO 18
                    GA 23
                    HA 24
                    ES 14
                    OURS 15}

ChargingTime2018 = {NC 30
                    BA 13
                    PSO 19
                    GA 21
                    HA 29
                    ES 17
                    OURS 12}


In [99]:
TT, WT, CT = 15, 2, 17   # with RCS
TT, WT, CT = 15, 5, 35   # without RCS    With all chargers ignore waiting time

QoS = TT + WT + CT

Threshold = 50 - QoS # 50 is the maximum time to complete travelling, charging and waiting

QoS

34

79